Install and Import Dependencies 

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1MB 1.3MB/s eta 0:14:18tcmalloc: large alloc 1147494400 bytes == 0x558db6572000 @  0x7f4a18859615 0x558d7d9e8cdc 0x558d7dac852a 0x558d7d9ebafd 0x558d7dadcfed 0x558d7da5f988 0x558d7da5a4ae 0x558d7d9ed3ea 0x558d7da5f7f0 0x558d7da5a4ae 0x558d7d9ed3ea 0x558d7da5c32a 0x558d7dadde36 0x558d7da5b853 0x558d7dadde36 0x558d7da5b853 0x558d7dadde36 0x558d7da5b853 0x558d7dadde36 0x558d7db603e1 0x558d7dac06a9 0x558d7da2bcc4 0x558d7d9ec559 0x558d7da604f8 0x558d7d9ed30a 0x558d7da5b3b5 0x558d7da5a7ad 0x558d7d9ed3ea 0x558d7da5b3b5 0x558d7d9ed30a 0x558d7da5b3b5
     |█████████████████               | 1055.7MB 1.4MB/s eta 0:11:10tcmalloc: large alloc 1434370048 bytes == 0x558dfabc8000 @  0x7f4a18859615 0x558d7d9e8cdc 0x558d7dac852a 0x558d7d9ebafd 0x558d7dadcfed 0x558d7da5f988 0x558d7da5a4ae 0x558d7d9ed3ea 0x558d7da5f7f0 0x558d7da5a4ae 0x558d7d9ed3ea 0x558d7da5c32a 0x558d7dadde36 0x558d7da5b853 0

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 2.3MB 31.9MB/s 
     |████████████████████████████████| 3.3MB 45.6MB/s 
     |████████████████████████████████| 901kB 33.6MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

Instantiate Model 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Encode and Calculate Sentiment 

In [ ]:
token = tokenizer.encode('Why are you such a dick', return_tensors = 'pt')

In [ ]:
token[0]

tensor([  101, 18469, 10320, 10855, 11165,   143, 19715,   102])

In [ ]:
tokenizer.decode(token[0])

'[CLS] why are you such a dick [SEP]'

In [ ]:
result = model(token)
result.logits 

tensor([[ 1.3600,  0.6482, -0.1948, -1.0619, -0.6072]],
       grad_fn=<AddmmBackward>)

In [ ]:
result 

SequenceClassifierOutput([('logits',
                           tensor([[ 1.3600,  0.6482, -0.1948, -1.0619, -0.6072]],
                                  grad_fn=<AddmmBackward>))])

In [ ]:
int(torch.argmax(result.logits))+1

1

Collect Reviews 

In [ ]:
# using request library to grab the webpage and getting a response code ( we can also check with this by r.text= everything that comprises the webpage)
r = requests.get('https://www.tripadvisor.com/ShowUserReviews-g304552-d1202019-r206352713-The_Restaurant-Shimla_Shimla_District_Himachal_Pradesh.html')
#passing the r.text to our BeautifulSoup with the setting parser type HTML 
soup = BeautifulSoup(r.text, 'html.parser')
# looking for anything named 'partial_entry' which is in the class using regex
regex = re.compile('.*partial_entry.*')
#using find_all to find all the tag (p) that matches our 'regex' within the 'soup'
results = soup.find_all('p', {'class':regex})
#results[0].text
#we are including each text in the results in a list with removed tags!
reviews = [result.text for result in results]

In [ ]:
reviews

['The welcome, attention and willingness to please and make customers happy is possibly the best ever example of customer service I have ever experienced. I loved this hotel so nostalgic and beautiful. Chennai in the spa was amazing - I had a fantastic massage. Food quality and service excellent. Pool and spa excellent. The staff all excellent at every level unlike far surpassing that of its rivals. Near the beautiful old town of a Shimla. I was there for 3 days but honestly would have liked a week. Thank you one and all for an amazingly memorable and comfortable stay.',
 'Dear Guest Greetings from the hills. Many Thanks for appreciating Food and service in the hotel, I have also passed on your compliments to our Pool and Spa team.The whole team is delighted to heart the sane.Looking forward to see you soonThank you and Best RegardsAnupam GulatiExecutive Chef',
 'I and my wife had nice dinner here with our small daughter. We ordered soup and noodles with red wine. Food was awesome and 

Loading the review into DataFrame and Score 

In [ ]:
#  a gentle reading is required 
# https://www.octoparse.com/blog/web-scraping-using-python

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
df = pd.DataFrame(np.array(reviews), columns = ['review'])

In [ ]:
df

,review
0,"The welcome, attention and willingness to plea..."
1,Dear Guest Greetings from the hills. Many Than...
2,I and my wife had nice dinner here with our sm...
3,"Dear Guest,It is indeed our pleasure to know t..."
4,My husband and I with our daughter had dinner ...
5,"Dear Guest,It is indeed our pleasure to know t..."
6,Huge selection of mouthwatering Indian and Int...
7,Dear GuestWe are delighted to know that you en...
8,"My wife and I had a falafel and coffee here, b..."
9,"Dear Guest,Greetings from Shimla, the whole te..."


In [ ]:
df['review'].iloc[2]

'I and my wife had nice dinner here with our small daughter. We ordered soup and noodles with red wine. Food was awesome and staff was quick and courteous. But the best part was the colonial architecture of restaurant, furniture and big window panes. You kind of taken to the past and enjoyed. It has good value of money. The staff was friendly.'

In [ ]:
# making a function that will pass each and every 'partial_entry' into our sentiment analyser (encode and giving us the score)

def sentiment_score(review):
  token = tokenizer.encode(review, return_tensors='pt')  #why pt
  result = model(token)
  return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[2])

4

In [ ]:
df['sentiment']= df['review'].apply(lambda x: sentiment_score(x))

In [ ]:
df

,review,sentiment
0,"The welcome, attention and willingness to plea...",5
1,Dear Guest Greetings from the hills. Many Than...,5
2,I and my wife had nice dinner here with our sm...,4
3,"Dear Guest,It is indeed our pleasure to know t...",5
4,My husband and I with our daughter had dinner ...,5
5,"Dear Guest,It is indeed our pleasure to know t...",5
6,Huge selection of mouthwatering Indian and Int...,5
7,Dear GuestWe are delighted to know that you en...,5
8,"My wife and I had a falafel and coffee here, b...",4
9,"Dear Guest,Greetings from Shimla, the whole te...",5
